<a href="https://colab.research.google.com/github/yudhiesh1997/Housing-Prices-Advanced-Regression/blob/master/Non_linearRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import sklearn 
import pandas as pd
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
X_train = pd.read_csv("/content/drive/My Drive/house-prices-advanced-regression-techniques (1)/X_train.csv")
X_test = pd.read_csv("/content/drive/My Drive/house-prices-advanced-regression-techniques (1)/X_test.csv")
y_train = pd.read_csv("/content/drive/My Drive/house-prices-advanced-regression-techniques (1)/y_train.csv")

In [3]:
X_train.head()

,MSZoning,LotArea,LotShape,BldgType,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,KitchenQual,Functional,Fireplaces,GarageType,GarageFinish,GarageCars,GarageCond,PavedDrive,WoodDeckSF,SaleCondition
0,0.5,0.366344,1.0,0.0,0.666667,0.500,0.036765,0.098361,0.0,0.333333,1.0,0.5,1.00,1.00,0.333333,0.0,1.0,0.356155,0.577712,0.333333,0.666667,0.5,0.666667,1.0,0.000000,0.0,0.666667,0.50,1.0,1.0,0.000000,0.5
1,0.5,0.391317,1.0,0.0,0.555556,0.875,0.227941,0.524590,0.0,1.000000,1.0,0.5,1.00,0.25,0.000000,0.0,1.0,0.503056,0.470245,0.000000,0.666667,0.0,1.000000,1.0,0.333333,0.0,0.666667,0.50,1.0,1.0,0.347725,0.5
2,0.5,0.422359,0.0,0.0,0.666667,0.500,0.051471,0.114754,0.0,0.333333,1.0,0.5,1.00,0.75,0.333333,0.0,1.0,0.383441,0.593095,0.333333,0.666667,0.5,0.666667,1.0,0.333333,0.0,0.666667,0.50,1.0,1.0,0.000000,0.5
3,0.5,0.390295,0.0,0.0,0.666667,0.500,0.669118,0.606557,0.0,1.000000,1.0,1.0,0.25,1.00,0.000000,0.5,1.0,0.399941,0.579157,0.333333,0.333333,0.0,0.666667,1.0,0.333333,0.6,1.000000,0.75,1.0,1.0,0.000000,0.0
4,0.5,0.468761,0.0,0.0,0.777778,0.500,0.058824,0.147541,0.0,0.333333,1.0,0.5,1.00,0.00,0.333333,0.0,1.0,0.466237,0.666523,0.333333,0.666667,0.5,0.666667,1.0,0.333333,0.0,0.666667,0.75,1.0,1.0,0.224037,0.5


In [4]:
X_test.head()

,MSZoning,LotArea,LotShape,BldgType,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,ExterQual,ExterCond,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,HeatingQC,CentralAir,1stFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,KitchenQual,Functional,Fireplaces,GarageType,GarageFinish,GarageCars,GarageCond,PavedDrive,WoodDeckSF,SaleCondition
0,0.6,0.566360,1.0,0.0,0.444444,0.625,0.384615,0.822581,0.2,1.000000,1.0,1.0,1.0,1.0,0.833333,1.0,1.0,0.312253,0.312253,0.0,0.25,0.0,1.0,1.0,0.00,0.166667,1.000000,0.2,1.0,1.0,0.098315,0.8
1,0.8,0.622527,0.0,0.0,0.555556,0.625,0.407692,0.870968,0.6,1.000000,1.0,1.0,1.0,1.0,0.000000,1.0,1.0,0.468253,0.468253,0.0,0.25,0.5,0.5,1.0,0.00,0.166667,1.000000,0.2,1.0,1.0,0.275983,0.8
2,0.8,0.614005,0.0,0.0,0.444444,0.500,0.107692,0.225806,0.2,1.000000,1.0,0.5,1.0,1.0,0.333333,0.5,1.0,0.326139,0.548792,0.0,0.50,0.5,1.0,1.0,0.25,0.166667,0.000000,0.4,1.0,1.0,0.148876,0.8
3,0.8,0.524583,0.0,0.0,0.555556,0.625,0.100000,0.225806,0.2,1.000000,1.0,1.0,1.0,1.0,0.333333,0.0,1.0,0.325285,0.542672,0.0,0.50,0.5,0.5,1.0,0.25,0.166667,0.000000,0.4,1.0,1.0,0.252809,0.8
4,0.8,0.335596,0.0,1.0,0.777778,0.500,0.146154,0.322581,0.2,0.666667,1.0,0.5,1.0,1.0,0.000000,0.0,1.0,0.453388,0.453388,0.0,0.50,0.0,0.5,1.0,0.00,0.166667,0.666667,0.4,1.0,1.0,0.000000,0.8


In [5]:
y_train.head()

,SalePrice
0,12.247694
1,12.109011
2,12.317167
3,11.849398
4,12.429216


## Model Selection: Non-linear Regression 

For this I will be using multiple models where I will train and find the best hyperparameters for them and them showcase the best one for this dataset. 

1. Linear Support Vector Machine 
2. Decision Trees
3. Random Forest 
4. AdaBoost
5. XGBoost
6. Blending of models 

In [6]:
X_train_ = X_train.values
X_test_ = X_test.values 
y_train_ = y_train.values 

In [7]:
X_train_.shape

(1460, 32)

In [8]:
X_test.isnull().sum()

MSZoning         0
LotArea          0
LotShape         0
BldgType         0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
RoofStyle        0
ExterQual        0
ExterCond        0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
HeatingQC        0
CentralAir       0
1stFlrSF         0
GrLivArea        0
BsmtFullBath     0
FullBath         0
HalfBath         0
KitchenQual      0
Functional       0
Fireplaces       0
GarageType       0
GarageFinish     0
GarageCars       0
GarageCond       0
PavedDrive       0
WoodDeckSF       0
SaleCondition    0
dtype: int64

In [9]:
X_train_.reshape(-1,1)
X_test_.reshape(-1,1)
y_train_.reshape(-1,1)

array([[12.24769432],
       [12.10901093],
       [12.31716669],
       ...,
       [12.49312952],
       [11.86446223],
       [11.90158345]])

In [10]:
X_test_.shape

(1459, 32)

In [11]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(X_train_)
    rmse= np.sqrt(-cross_val_score(model, X_train_, y_train_, scoring="neg_mean_squared_error", cv = kf))
    return rmse 

In [12]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Kernel Ridge score: 0.1354 (0.0137)



In [13]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
score = rmsle_cv(lasso)
print("Lasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

Lasso score: 0.1338 (0.0130)



In [14]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

ElasticNet score: 0.1337 (0.0130)



In [15]:
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)
score = rmsle_cv(GBoost)
print("Gradient Boosting score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
 

Gradient Boosting score: 0.1310 (0.0132)



In [16]:
LassoMd = lasso.fit(X_train_,y_train)
ENetMd = ENet.fit(X_train_,y_train)
KRRMd = KRR.fit(X_train_,y_train)
GBoostMd = GBoost.fit(X_train_,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/_gb.py:1454: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
finalMd = (np.expm1(LassoMd.predict(X_test_)) + np.expm1(ENetMd.predict(X_test_)) + np.expm1(KRRMd.predict(X_test_)) + np.expm1(GBoostMd.predict(X_test_)) ) / 4
finalMd.shape

(1459, 1459)